In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import regex

# below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML
%matplotlib inline

In [2]:
# extra imports
sns.set_context('poster')

 #### Load Pickle

In [3]:
with open('pia_df.pickle', 'rb') as read_file:
    df = pickle.load(read_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62766 entries, 0 to 62765
Data columns (total 33 columns):
subject_id              61532 non-null float64
hadm_id                 61532 non-null float64
icustay_id              61532 non-null float64
dbsource                61532 non-null object
first_careunit          61532 non-null object
last_careunit           61532 non-null object
first_wardid            61532 non-null float64
last_wardid             61532 non-null float64
intime                  61532 non-null datetime64[ns]
outtime                 61522 non-null datetime64[ns]
los                     61522 non-null float64
admittime               62722 non-null datetime64[ns]
dischtime               62722 non-null datetime64[ns]
deathtime               6650 non-null datetime64[ns]
admission_type          62722 non-null object
admission_location      62722 non-null object
discharge_location      62722 non-null object
insurance               62722 non-null object
language          

**Some notes on this table:** 

* icustay_id is unique to this table

* There are more stays than there are patients - 78.8% of the visits are single visits. 

*  hadm_id is not unique for this table -- so some people came in and out of the ICU (57786/61532 = 93.9% unique visits)

* There are 3 db sources: 
    
        carevue       37776
        metavision    23620
        both            136

* 6 first careunits & same last careunits: 
    
        CareUnit first    last
        MICU     21088    21310
        CSRU      9312    9392
        SICU      8891    9213
        NICU      8100    8100
        CCU       7726    7406
        TSICU     6415    6111
    -- how many people switched? And where did they switch to? 
    
    -- Perhaps features: switch yes/no - switch to X, switch to Y .. etc
    
    
* first ward_id - 16 of them
* last_ward_id - 17 of them --- ids are #'s -- reference another table 
        
* 61531 unique intimes & 61518 unique outimes
* 40175 unique los - Length of stay (measured in fractional days)


### Cleaning to run an MVP Model 

In [4]:
df.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'dbsource', 'first_careunit',
       'last_careunit', 'first_wardid', 'last_wardid', 'intime', 'outtime',
       'los', 'admittime', 'dischtime', 'deathtime', 'admission_type',
       'admission_location', 'discharge_location', 'insurance', 'language',
       'religion', 'marital_status', 'ethnicity', 'edregtime', 'edouttime',
       'diagnosis', 'hospital_expire_flag', 'has_chartevents_data', 'gender',
       'dob', 'dod', 'dod_hosp', 'dod_ssn', 'expire_flag'],
      dtype='object')

In [5]:
# REMOVE NICU

df = df[df.first_careunit != 'NICU']
df = df[df.last_careunit != 'NICU']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54666 entries, 0 to 62765
Data columns (total 33 columns):
subject_id              53432 non-null float64
hadm_id                 53432 non-null float64
icustay_id              53432 non-null float64
dbsource                53432 non-null object
first_careunit          53432 non-null object
last_careunit           53432 non-null object
first_wardid            53432 non-null float64
last_wardid             53432 non-null float64
intime                  53432 non-null datetime64[ns]
outtime                 53429 non-null datetime64[ns]
los                     53429 non-null float64
admittime               54622 non-null datetime64[ns]
dischtime               54622 non-null datetime64[ns]
deathtime               6586 non-null datetime64[ns]
admission_type          54622 non-null object
admission_location      54622 non-null object
discharge_location      54622 non-null object
insurance               54622 non-null object
language          

In [6]:
# Create new column - time prior to being admitted to ICU
# admittime - intime
df['time_before_ICU'] = df.intime - df.admittime

In [7]:
df.time_before_ICU.head()

0     7 days 03:02:12
1     0 days 02:48:04
2     0 days 00:01:14
3     0 days 00:01:37
4   -1 days +23:49:32
Name: time_before_ICU, dtype: timedelta64[ns]

In [8]:
df['prior_ICU_hours'] = df.time_before_ICU.apply(lambda x: round((((x.days)*24*60*60)+x.seconds)/360,1))
df.prior_ICU_hours.head()

0    1710.4
1      28.0
2       0.2
3       0.3
4      -1.7
Name: prior_ICU_hours, dtype: float64

In [9]:
# Create new column - time spent in emergency department
df['ER_time'] = df.edouttime - df.edregtime
df.ER_time.head()

0   06:47:00
1        NaT
2   05:01:00
3   04:47:00
4        NaT
Name: ER_time, dtype: timedelta64[ns]

In [10]:
df[['los','ER_time']].head()

,los,ER_time
0,3.0786,06:47:00
1,5.8884,NaT
2,1.3194,05:01:00
3,1.2914,04:47:00
4,2.1549,NaT


In [11]:
print(df.ER_time[48287])
print(df.ER_time[48287].seconds)


2 days 22:28:00
80880


In [12]:
df['ER_hours'] = df.ER_time.apply(lambda x: round((((x.days)*24*60*60)+x.seconds)/360,1))

In [13]:
# AGE
df['age_timedelta'] = df.admittime - df.dob
df.age_timedelta.head()

0   22298 days 16:04:00
1   17639 days 17:02:00
2   19749 days 11:27:00
3   15831 days 10:02:00
4   17369 days 10:45:00
Name: age_timedelta, dtype: timedelta64[ns]

In [14]:
# Dropped columns without hadm id
df = df[df.hadm_id.isnull() == False]

In [15]:
df[df.age_timedelta.isnull()]

,subject_id,hadm_id,icustay_id,dbsource,first_careunit,last_careunit,first_wardid,last_wardid,intime,outtime,...,dob,dod,dod_hosp,dod_ssn,expire_flag,time_before_ICU,prior_ICU_hours,ER_time,ER_hours,age_timedelta


In [16]:
# convert timedelta objects to floats
#round(df.age[0].days/365)
df['age_years'] = df.age_timedelta.map(lambda x: round(x.days/365))
df['age_years'].head()

0    61
1    48
2    54
3    43
4    48
Name: age_years, dtype: int64

In [17]:
df.marital_status.value_counts()

MARRIED              25472
SINGLE               13560
WIDOWED               7545
DIVORCED              3385
SEPARATED              600
UNKNOWN (DEFAULT)      371
LIFE PARTNER            16
Name: marital_status, dtype: int64

In [18]:
df['marital_status_bin'] = df.marital_status

In [19]:
df['marital_status_bin'][df.marital_status_bin == 'LIFE PARTNER'] = 'MARRIED'
df['marital_status_bin'].value_counts()

C:\Users\kmuss\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


MARRIED              25488
SINGLE               13560
WIDOWED               7545
DIVORCED              3385
SEPARATED              600
UNKNOWN (DEFAULT)      371
Name: marital_status_bin, dtype: int64

In [20]:
df['marital_status_bin'][df.marital_status_bin == 'WIDOWED'] = 'SINGLE'
df['marital_status_bin'][df.marital_status_bin == 'DIVORCED'] = 'SINGLE'
df['marital_status_bin'][df.marital_status_bin == 'SEPARATED'] = 'SINGLE'
df['marital_status_bin'].value_counts()

C:\Users\kmuss\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\kmuss\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\kmuss\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


MARRIED              25488
SINGLE               25090
UNKNOWN (DEFAULT)      371
Name: marital_status_bin, dtype: int64

In [21]:
df.admission_location.value_counts()

EMERGENCY ROOM ADMIT         24121
CLINIC REFERRAL/PREMATURE    10631
PHYS REFERRAL/NORMAL DELI     9171
TRANSFER FROM HOSP/EXTRAM     9122
TRANSFER FROM SKILLED NUR      290
TRANSFER FROM OTHER HEALT       86
TRSF WITHIN THIS FACILITY        5
** INFO NOT AVAILABLE **         5
HMO REFERRAL/SICK                1
Name: admission_location, dtype: int64

In [27]:
# BIN LAST 5 ENTRIES into OTHER

df['admission_location'][df.admission_location.str.contains('TRANSFER FROM SKILLED NUR')] = 'OTHER'
df['admission_location'][df.admission_location.str.contains('TRANSFER FROM OTHER HEALT')] = 'OTHER'
df['admission_location'][df.admission_location == 'TRSF WITHIN THIS FACILITY'] = 'OTHER'
df['admission_location'][df.admission_location == '** INFO NOT AVAILABLE **'] = 'OTHER'
df['admission_location'][df.admission_location == 'HMO REFERRAL/SICK'] = 'OTHER'

df.admission_location.value_counts()

C:\Users\kmuss\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kmuss\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\kmuss\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\kmuss\Anaconda3\l

EMERGENCY ROOM ADMIT         24121
CLINIC REFERRAL/PREMATURE    10631
PHYS REFERRAL/NORMAL DELI     9171
TRANSFER FROM HOSP/EXTRAM     9122
OTHER                          387
Name: admission_location, dtype: int64

In [39]:
df.language.value_counts()

ENGL    30421
SPAN     1126
RUSS      837
PTUN      637
CANT      395
PORT      347
CAPE      233
HAIT      154
ITAL      132
MAND      127
VIET       87
GREE       80
ARAB       49
PERS       47
CAMB       46
POLI       35
AMER       34
HIND       27
KORE       24
ALBA       18
FREN       16
SOMA       15
THAI       14
ETHI       12
*ARM       11
*YID        8
*HUN        8
*CHI        7
LAOT        7
*GUJ        7
        ...  
*CAN        2
TURK        2
* FU        2
*TOI        2
*FUL        2
* BE        2
*TOY        2
*AMH        2
BENG        2
SERB        2
**SH        2
*LEB        2
*YOR        1
*DEA        1
*RUS        1
GERM        1
*FIL        1
*ROM        1
*CRE        1
*PER        1
*PHI        1
*BOS        1
*LIT        1
** T        1
*ARA        1
*PUN        1
*FAR        1
*TAM        1
*SPA        1
*NEP        1
Name: language, Length: 75, dtype: int64

In [54]:
df.language[0]

'ENGL'

In [56]:
df.language_bin[0]

'ENGL'

In [52]:
if "ENGL" in df.language[0]: 
    print('yes')
    

yes


In [57]:
df.language_bin.isna().sum()

18370

In [58]:
# Assume empty columns speak English
df.language_bin = df.language

def speaks_english(x):
    if x is None: 
        return 1
    elif "ENGL" in x: 
        return 1
    else: 
        return 0
df['language_bin'] = df['language_bin'].apply(speaks_english)

In [59]:
df.language_bin.value_counts()

1    48791
0     4641
Name: language_bin, dtype: int64

In [77]:
df.religion.value_counts()

CATHOLIC                  19464
NOT SPECIFIED             10759
PROTESTANT QUAKER          6974
UNOBTAINABLE               5681
JEWISH                     5100
OTHER                      2523
EPISCOPALIAN                733
GREEK ORTHODOX              441
CHRISTIAN SCIENTIST         348
BUDDHIST                    227
MUSLIM                      192
JEHOVAH'S WITNESS           134
UNITARIAN-UNIVERSALIST      111
ROMANIAN EAST. ORTH          76
7TH DAY ADVENTIST            73
HINDU                        70
BAPTIST                      28
HEBREW                       18
METHODIST                     8
LUTHERAN                      1
Name: religion, dtype: int64

In [110]:
# Note - we can only know if they are relgious or unknown
## There are no categories for non-religious -- just "not specified"

#df['religion_bin'] = df.religion


def is_religious(x):
    if x is None: 
        return 0
    elif x == "NOT SPECIFIED": 
        return 0
    elif x == "UNOBTAINABLE": 
        return 0
    else: 
        return 1
df['religion_2'] = df['religion'].apply(is_religious)

In [112]:
df.religion_2.value_counts()

1    36521
0    16911
Name: religion_2, dtype: int64

In [111]:
df.religion_2.head()

0    1
1    1
2    0
3    1
4    1
Name: religion_2, dtype: int64

In [114]:
df.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'dbsource', 'first_careunit',
       'last_careunit', 'first_wardid', 'last_wardid', 'intime', 'outtime',
       'los', 'admittime', 'dischtime', 'deathtime', 'admission_type',
       'admission_location', 'discharge_location', 'insurance', 'language',
       'religion', 'marital_status', 'ethnicity', 'edregtime', 'edouttime',
       'diagnosis', 'hospital_expire_flag', 'has_chartevents_data', 'gender',
       'dob', 'dod', 'dod_hosp', 'dod_ssn', 'expire_flag', 'time_before_ICU',
       'prior_ICU_hours', 'ER_time', 'ER_hours', 'age_timedelta', 'age_years',
       'marital_status_bin', 'language_bin', 'religion_bin', 'religion_2'],
      dtype='object')

In [115]:
# Admission type
df['admission_type'].value_counts()

EMERGENCY    44584
ELECTIVE      7451
URGENT        1397
Name: admission_type, dtype: int64

In [116]:
# Ethnicity 
df['ethnicity'].value_counts()

WHITE                                                       37912
BLACK/AFRICAN AMERICAN                                       4752
UNKNOWN/NOT SPECIFIED                                        4305
HISPANIC OR LATINO                                           1378
OTHER                                                        1178
UNABLE TO OBTAIN                                              846
ASIAN                                                         778
PATIENT DECLINED TO ANSWER                                    457
ASIAN - CHINESE                                               242
HISPANIC/LATINO - PUERTO RICAN                                232
BLACK/CAPE VERDEAN                                            182
WHITE - RUSSIAN                                               168
MULTI RACE ETHNICITY                                          111
BLACK/HAITIAN                                                 101
ASIAN - ASIAN INDIAN                                           89
HISPANIC/L

In [118]:
# Function attribution - https://github.com/daniel-codes/hospital-los-predictor/blob/master/hospital_los_prediction.ipynb 

df['ethnicity'].replace(regex=r'^ASIAN\D*', value='ASIAN', inplace=True)
df['ethnicity'].replace(regex=r'^WHITE\D*', value='WHITE', inplace=True)
df['ethnicity'].replace(regex=r'^HISPANIC\D*', value='HISPANIC/LATINO', inplace=True)
df['ethnicity'].replace(regex=r'^BLACK\D*', value='BLACK/AFRICAN AMERICAN', inplace=True)
df['ethnicity'].replace(['UNABLE TO OBTAIN', 'OTHER', 'PATIENT DECLINED TO ANSWER', 
                         'UNKNOWN/NOT SPECIFIED'], value='OTHER/UNKNOWN', inplace=True)
df['ethnicity'].loc[~df['ethnicity'].isin(df['ethnicity'].value_counts().nlargest(5).index.tolist())] = 'OTHER/UNKNOWN'
df['ethnicity'].value_counts()

C:\Users\kmuss\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


WHITE                     38248
OTHER/UNKNOWN              7065
BLACK/AFRICAN AMERICAN     5075
HISPANIC/LATINO            1807
ASIAN                      1237
Name: ethnicity, dtype: int64

In [119]:
with open('temp_redo_pia_df.pickle', 'wb') as to_write:
    pickle.dump(df, to_write)

In [121]:
df.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'dbsource', 'first_careunit',
       'last_careunit', 'first_wardid', 'last_wardid', 'intime', 'outtime',
       'los', 'admittime', 'dischtime', 'deathtime', 'admission_type',
       'admission_location', 'discharge_location', 'insurance', 'language',
       'religion', 'marital_status', 'ethnicity', 'edregtime', 'edouttime',
       'diagnosis', 'hospital_expire_flag', 'has_chartevents_data', 'gender',
       'dob', 'dod', 'dod_hosp', 'dod_ssn', 'expire_flag', 'time_before_ICU',
       'prior_ICU_hours', 'ER_time', 'ER_hours', 'age_timedelta', 'age_years',
       'marital_status_bin', 'language_bin', 'religion_bin', 'religion_2'],
      dtype='object')

# Dummy variables

* religion_2
* marital_status_bin
* ethnicity
* language_bin
* admission_type
* admission_location
* first_careunit
* insurance
* gender

In [125]:
df_dummies = pd.get_dummies(df, columns=(['marital_status_bin',
                                 'religion_2', 
                                 'ethnicity',
                                 'language_bin',
                                 'admission_type',
                                 'admission_location',
                                 'first_careunit',
                                 'insurance',
                                 'gender',
                                ]))
#df_dummies.head()

In [126]:
df_dummies.head()

,subject_id,hadm_id,icustay_id,dbsource,last_careunit,first_wardid,last_wardid,intime,outtime,los,...,first_careunit_MICU,first_careunit_SICU,first_careunit_TSICU,insurance_Government,insurance_Medicaid,insurance_Medicare,insurance_Private,insurance_Self Pay,gender_F,gender_M
0,55973.0,152234.0,200001.0,metavision,MICU,23.0,23.0,2181-11-25 19:06:12,2181-11-28 20:59:25,3.0786,...,1,0,0,0,0,1,0,0,1,0
1,27513.0,163557.0,200003.0,carevue,SICU,57.0,57.0,2199-08-02 19:50:04,2199-08-08 17:09:18,5.8884,...,0,1,0,0,0,0,1,0,0,1
2,10950.0,189514.0,200006.0,carevue,MICU,52.0,52.0,2159-09-03 11:28:14,2159-09-04 19:08:10,1.3194,...,1,0,0,0,1,0,0,0,0,1
3,20707.0,129310.0,200007.0,carevue,CCU,57.0,57.0,2109-02-17 10:03:37,2109-02-18 17:03:12,1.2914,...,0,0,0,0,0,0,1,0,0,1
4,29904.0,129607.0,200009.0,carevue,CSRU,15.0,15.0,2189-11-30 10:34:32,2189-12-02 14:17:37,2.1549,...,0,0,0,0,0,0,1,0,1,0


#### Remove one column from each category to remove co-linearity

* marital_status_bin_UNKNOWN (DEFAULT)
* religion_2_0 (not religious/ unknown)
* ethnicity_OTHER/UNKNOWN
* language_bin_1 (speaks English)
* admission_type_ELECTIVE
* admission_location_OTHER
* first_careunit_TSICU
* insurance_Government
* gender_M

In [130]:
df_dummies.drop(columns=(['marital_status_bin_UNKNOWN (DEFAULT)',
                          'religion_2_0',
                          'ethnicity_OTHER/UNKNOWN',
                          'language_bin_1',
                          'admission_type_ELECTIVE',
                          'admission_location_OTHER',
                          'first_careunit_TSICU',
                          'insurance_Government',
                          'gender_M']),axis=1, inplace=True)
                          

In [131]:
df_dummies.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'dbsource', 'last_careunit',
       'first_wardid', 'last_wardid', 'intime', 'outtime', 'los', 'admittime',
       'dischtime', 'deathtime', 'discharge_location', 'language', 'religion',
       'marital_status', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'dob', 'dod',
       'dod_hosp', 'dod_ssn', 'expire_flag', 'time_before_ICU',
       'prior_ICU_hours', 'ER_time', 'ER_hours', 'age_timedelta', 'age_years',
       'religion_bin', 'marital_status_bin_MARRIED',
       'marital_status_bin_SINGLE', 'religion_2_1', 'ethnicity_ASIAN',
       'ethnicity_BLACK/AFRICAN AMERICAN', 'ethnicity_HISPANIC/LATINO',
       'ethnicity_WHITE', 'language_bin_0', 'admission_type_EMERGENCY',
       'admission_type_URGENT', 'admission_location_CLINIC REFERRAL/PREMATURE',
       'admission_location_EMERGENCY ROOM ADMIT',
       'admission_location_PHYS REFERRAL/NORMAL DELI',
       'admission_location_TRANSFER FR